In [2]:
import os
import cv2
import glob
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import itertools
from sklearn.metrics import confusion_matrix


In [3]:
# Ignore the warning
warnings.filterwarnings("ignore")
# Auto-reload extensions
%reload_ext autoreload
%autoreload 2
# Display Matplotlib plots inline
%matplotlib inline

In [ ]:
##### Load the data from zipfile locally#####

# import zipfile

# zip_file_path = os.path.join('..', 'raw_data', 'archive.zip')
# extract_dir = os.path.join('..', 'raw_data', 'cnn_data')

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# extracted_files = os.listdir(extract_dir)
# print("Extracted files:", extracted_files)

# Data Exploration
1. remove the Grayscale images
2. resize images
3. normalization
4. argumentation

## Remove Grayscale images

In [4]:
styles = [
    'Achaemenid architecture',
    'American craftsman style',
    'American Foursquare architecture',
    'Ancient Egyptian architecture',
    'Art Deco architecture',
    'Art Nouveau architecture',
    'Baroque architecture',
    'Bauhaus architecture',
    'Beaux-Arts architecture',
    'Byzantine architecture',
    'Chicago school architecture',
    'Colonial architecture',
    'Deconstructivism',
    'Edwardian architecture',
    'Georgian architecture',
    'Gothic architecture',
    'Greek Revival architecture',
    'International style',
    'Novelty architecture',
    'Palladian architecture',
    'Postmodern architecture',
    'Queen Anne architecture',
    'Romanesque architecture',
    'Russian Revival architecture',
    'Tudor Revival architecture'
]

In [5]:
from PIL import Image

base_folder_path = '../raw_data/cnn_data/architectural-styles-dataset'

def black_white(image_path):
    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image)

    if np.all(image_array[:, :, 0] == image_array[:, :, 1]) and  np.all(image_array[:, :, 1]== image_array[:, :, 2]):
        return True
    else:
        return False


def process_images_in_style(style_name):
    folder_path = os.path.join(base_folder_path, style_name)
    deleted_count = 0
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return

    for file_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, file_name)
        if black_white(image_path):
            os.remove(image_path)
            deleted_count += 1
#             print(f"Grayscale image found: {file_name} in {style_name}")

    print(f"Deleted {deleted_count} files in the {style_name}")


In [6]:
# Process images for each folder
for style in styles:
    print(f"Processing images for style: {style}")
    process_images_in_style(style)

Deleted 0 files in the Art Nouveau architecture
Processing images for style: Baroque architecture
Deleted 0 files in the Baroque architecture
Processing images for style: Bauhaus architecture
Deleted 0 files in the Bauhaus architecture
Processing images for style: Beaux-Arts architecture
Deleted 0 files in the Beaux-Arts architecture
Processing images for style: Byzantine architecture
Deleted 0 files in the Byzantine architecture
Processing images for style: Chicago school architecture
Deleted 0 files in the Chicago school architecture
Processing images for style: Colonial architecture
Deleted 0 files in the Colonial architecture
Processing images for style: Deconstructivism
Deleted 0 files in the Deconstructivism
Processing images for style: Edwardian architecture
Deleted 0 files in the Edwardian architecture
Processing images for style: Georgian architecture
Deleted 0 files in the Georgian architecture
Processing images for style: Gothic architecture
Deleted 0 files in the Gothic arc

## Split the dataset into Train/Val/Test

In [7]:
import shutil
import random

# Define the base directory where the data is currently located
base_dir = '../raw_data/cnn_data/architectural-styles-dataset'
# Define the new base directory where the train/val/test folders will be created
new_base_dir = '../raw_data/cnn_data'

# Create directories for train, val, and test sets
train_dir = os.path.join(new_base_dir, 'train')
val_dir = os.path.join(new_base_dir, 'val')
test_dir = os.path.join(new_base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


# Function to split and copy files
def split_and_copy_files(style_name):
    source_folder = os.path.join(base_dir, style_name)
    if not os.path.exists(source_folder):
        print(f"Folder not found: {source_folder}")
        return

    files = os.listdir(source_folder)
    random.shuffle(files)

    # Calculate split indices
    total_files = len(files)
    train_count = int(0.7 * total_files)
    val_count = int(0.15 * total_files)

    train_files = files[:train_count]
    val_files = files[train_count:train_count + val_count]
    test_files = files[train_count + val_count:]

    # Create style-specific directories in train, val, and test folders
    train_style_dir = os.path.join(train_dir, style_name)
    val_style_dir = os.path.join(val_dir, style_name)
    test_style_dir = os.path.join(test_dir, style_name)

    os.makedirs(train_style_dir, exist_ok=True)
    os.makedirs(val_style_dir, exist_ok=True)
    os.makedirs(test_style_dir, exist_ok=True)

    # Copy files to respective directories
    for file_name in train_files:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(train_style_dir, file_name))

    for file_name in val_files:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(val_style_dir, file_name))

    for file_name in test_files:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(test_style_dir, file_name))

    print(f"Processed {style_name}: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")


# Split and copy files for each style
for style in styles:
    split_and_copy_files(style)

Processed Achaemenid architecture: 273 train, 58 val, 59 test
Processed American craftsman style: 254 train, 54 val, 56 test
Processed American Foursquare architecture: 251 train, 53 val, 55 test
Processed Ancient Egyptian architecture: 284 train, 60 val, 62 test
Processed Art Deco architecture: 389 train, 83 val, 84 test
Processed Art Nouveau architecture: 424 train, 91 val, 92 test
Processed Baroque architecture: 317 train, 68 val, 69 test
Processed Bauhaus architecture: 201 train, 43 val, 44 test
Processed Beaux-Arts architecture: 291 train, 62 val, 64 test
Processed Byzantine architecture: 216 train, 46 val, 47 test
Processed Chicago school architecture: 169 train, 36 val, 37 test
Processed Colonial architecture: 326 train, 70 val, 71 test
Processed Deconstructivism: 233 train, 49 val, 51 test
Processed Edwardian architecture: 195 train, 41 val, 43 test
Processed Georgian architecture: 264 train, 56 val, 58 test
Processed Gothic architecture: 227 train, 48 val, 50 test
Processed Gr

## Resize the Images

In [24]:
from tensorflow.keras.utils import image_dataset_from_directory

# Define the batch size
batch_size = 32

train_ds = image_dataset_from_directory(
  train_dir,
  labels = "inferred",
  label_mode = "int",
  seed=123,
  image_size=(150, 150),
  batch_size=batch_size)

# We define a second one for the test data
val_ds = image_dataset_from_directory(
  val_dir,
  labels = "inferred",
  label_mode = "int",
  seed=123,
  image_size=(150, 150),
  batch_size=batch_size)

test_ds = image_dataset_from_directory(
  test_dir,
  labels = "inferred",
  label_mode = "int",
  seed=123,
  image_size=(150, 150),
  batch_size=batch_size)

Found 6918 files belonging to 25 classes.
Found 1472 files belonging to 25 classes.
Found 1511 files belonging to 25 classes.


## Data Normalize and Augmentation

In [25]:
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom, RandomTranslation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.data import AUTOTUNE

# Define data augmentation layers
data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomTranslation(0.2, 0.2)
])

In [23]:
# Normalize the data
normalization_layer = Rescaling(1./255)

# Apply the normalization layer to the datasets
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)

# Prefetch the datasets for better performance
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Build and Train Model

In [26]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img



def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('accuracy')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [19]:
def plot_compare_history(history, name_history, history_1, name_history_1):

    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    ax[0].set_title('loss')

    ax[0].plot(history.epoch, history.history["loss"], label="Train loss " + name_history)
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss " + name_history)

    ax[0].plot(history_1.epoch, history_1.history["loss"], label="Train loss " + name_history_1)
    ax[0].plot(history_1.epoch, history_1.history["val_loss"], label="Validation loss " + name_history_1)

    ax[1].set_title('Accuracy')

    ax[1].plot(history.epoch, history.history["accuracy"], label="Train Accuracy " + name_history)
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation Accuracy " + name_history)

    ax[1].plot(history_1.epoch, history_1.history["accuracy"], label="Train Accuracy " + name_history_1)
    ax[1].plot(history_1.epoch, history_1.history["val_accuracy"], label="Validation Accuracy " + name_history_1)

    ax[0].legend()
    ax[1].legend()

## 1. VGG16 Model

In [27]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

base_model = VGG16(weights = "imagenet", include_top = False, input_shape = (150, 150, 3))

58889256/58889256 [==============================] - 5s 0us/step


In [33]:
inputs = layers.Input(shape = (150, 150, 3))

x = preprocess_input(inputs) # Then a preprocessing layer specifically designed for the VGG16
x = base_model(x) # Then our transfer learning model

x = layers.Flatten()(x) # Followed by our custom dense layers, tailored to our binary task

x = layers.Dense(128, activation = "relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation = "relu")(x)
x = layers.Dropout(0.3)(x)
pred = layers.Dense(25, activation = "softmax")(x)

# We use the keras Functional API to create our keras model

model_vgg = Model(inputs = inputs , outputs = pred)

# And we freeze the VGG16 model
base_model.trainable = False

In [34]:
model_vgg.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 tf.__operators__.getitem_1   (None, 150, 150, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_1 (TFOpLambd  (None, 150, 150, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 128)               1048

In [35]:
adam = optimizers.Adam(learning_rate = 0.001)
model_vgg.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [36]:
MODEL = "model_vgg"

modelCheckpoint = callbacks.ModelCheckpoint("{}.h5".format(MODEL), monitor="val_loss", verbose=0, save_best_only=True)

LRreducer = callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience=3, verbose=1, min_lr=0)

EarlyStopper = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

In [37]:
%%time
history_vgg = model_vgg.fit(
        train_ds,
        epochs=20,
        validation_data=val_ds,
        callbacks = [modelCheckpoint, LRreducer, EarlyStopper])

Epoch 1/20
217/217 [==============================] - 2092s 10s/step - loss: 4.1740 - accuracy: 0.1065 - val_loss: 2.8855 - val_accuracy: 0.1990 - lr: 0.0010
Epoch 2/20
114/217 [==============>...............] - ETA: 31:15 - loss: 3.0991 - accuracy: 0.1546